In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
! pip install mxnet
! pip install gluonts

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from datetime import datetime
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from gluonts.mx import DeepAREstimator, Trainer #, TrainerCallback, EarlyStoppingCallback
from gluonts.dataset.common import ListDataset

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [3]:
data = pd.read_csv('/content/gdrive/MyDrive/Thesis/newDeepAR/merged_df.csv')
data

,Date,ABBV,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2013-03-01,10.455062,7.5,232.282,1.984211,0.397212,3.337507,14.582205,11.414669,7.978126,4.851792,3.440210,-7.714279,9.067089,5.443922,-0.188640,-0.655,-0.2,0.069449
1,2013-04-01,12.922975,7.6,231.797,1.957500,0.085246,2.374537,3.881563,-3.568848,4.538222,-2.482836,7.359867,11.497419,3.537347,0.727651,-10.457790,-0.485,0.1,-0.026711
2,2013-05-01,-6.438455,7.5,231.893,1.759091,-0.190605,2.361004,-1.290451,16.816781,-1.231989,-4.008663,-0.638313,-8.792401,-2.711502,-6.329562,8.513136,0.096,-0.1,-0.198409
3,2013-06-01,-3.162299,7.5,232.445,1.928182,0.620501,-1.853781,-7.707318,-2.868894,2.762464,-6.788137,-0.535372,-3.805087,-1.463219,3.722003,2.269287,0.552,0.0,0.169091
4,2013-07-01,10.014486,7.3,232.900,2.300000,0.210359,5.724173,7.230424,-3.244595,8.898141,8.122938,4.655361,9.001745,1.272809,4.355584,17.540151,0.455,-0.2,0.371818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2023-01-01,-8.576205,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
119,2023-02-01,5.152609,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
120,2023-03-01,3.554254,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
121,2023-04-01,-5.176628,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [4]:
data = data.drop(columns=['unemployment_rate', 'CPI', 'treasury_yield'], axis=1)

In [5]:
def format(df):
  df = df.set_index('Date')
  return df

####################################################
def fit_n_predict(model, training, test):
  predictor = model.train(training)
  predictor = next(predictor.predict(test))
  samples = predictor.samples
  time = predictor.index
  return samples, time
####################################################
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)
    #print(df)

    t_loss = 0

    for i in df.index:
        t_loss += (alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i]) +
                 (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i])

    return t_loss
####################################################

## 1 Month

In [6]:
hyperparameters = {
    "prediction_length": 1,
    "context_length": 3,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

In [7]:
macro = data.copy()

for m in ['GDP_growth',	'SP500_return',	'inflation_change',	'unemp_change',	'treasury_yield_change']:
  macro[f'{m}_lag'] = macro[m].shift(1)
  macro[f'{m}_lag2'] = macro[m].shift(2)
  macro[f'{m}_lag3'] = macro[m].shift(3)


macro['Quater_dummy'] = 0
macro['Quater_rippel'] = 0
macro['Date'] = pd.to_datetime(macro['Date'])
for index, row in data.iterrows():
    date = pd.to_datetime(row['Date'])  # Convert the date string to datetime type
    if date.month in [3, 6, 9, 12]:
        macro.loc[index, 'Quater_dummy'] = 1
    if date.month in [1, 4, 7, 10]:
        macro.loc[index, 'Quater_rippel'] = 1

macro = format(macro)

In [6]:
dates1m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01']
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']
dates6m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01']
dates9m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01']
dates12m = ['2022-05-01']


In [9]:
def data_prep(df, ticker, horizon):
    train_data = ListDataset(
        [
            {
                "start": df.index[0],
                "target": df[ticker].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    test_data = ListDataset(
        [
            {
                "start": df.index[-horizon],
                "target": df[ticker][-horizon:].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    return train_data, test_data

In [11]:
def get_performance(ticker, dates, horizon):

  dta = macro.copy()
  t_loss = 0

  for d in dates:
    split_date = d
    train_df = dta[dta.index <= split_date]
    test_df = dta[dta.index > split_date]

    train, test = data_prep(train_df, ticker, horizon)
    predictions, check = fit_n_predict(estimator, train, test)
    p95 = np.percentile(predictions, 95, axis=0)
    p50 = np.percentile(predictions, 50, axis=0)
    p5 = np.percentile(predictions, 5, axis=0)

    var  = pd.Series(p5)
    agg = test_df[ticker]
    r = agg.values
    r = r[0:horizon]
    r = pd.Series(r)
    t_loss += tick_loss(0.05, r, var)

  combionation = 13-horizon
  t_loss = t_loss/combionation

  return t_loss

In [12]:
# 1month
abbv_1m_loss = get_performance('ABBV', dates1m, 1)
print(f'ABBV-1m: {abbv_1m_loss}')

100%|██████████| 50/50 [00:00<00:00, 160.34it/s, epoch=30/30, avg_epoch_loss=3.4]


ABBV-1m: 1.243734834336162


100%|██████████| 50/50 [00:00<00:00, 168.32it/s, epoch=30/30, avg_epoch_loss=3.37]

ABBV-3m: nan


In [13]:
hyperparameters = {
    "prediction_length": 3,
    "context_length": 4,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_3m_loss = get_performance('ABBV', dates3m, 3)
print(f'ABBV-3m: {abbv_3m_loss}')

100%|██████████| 50/50 [00:00<00:00, 98.87it/s, epoch=30/30, avg_epoch_loss=3.52]

ABBV-3m: 3.2616542883857136


In [14]:
hyperparameters = {
    "prediction_length": 6,
    "context_length": 9,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_6m_loss = get_performance('ABBV', dates6m, 6)
print(f'ABBV-6m: {abbv_6m_loss}')

100%|██████████| 50/50 [00:00<00:00, 63.90it/s, epoch=30/30, avg_epoch_loss=4.9]

ABBV-6m: 6.756454482459628


In [15]:
hyperparameters = {
    "prediction_length": 9,
    "context_length": 12,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_9m_loss = get_performance('ABBV', dates9m, 9)
print(f'ABBV-9m: {abbv_9m_loss}')

100%|██████████| 50/50 [00:00<00:00, 73.79it/s, epoch=30/30, avg_epoch_loss=5.91]

ABBV-9m: 12.414358351686513


In [16]:
hyperparameters = {
    "prediction_length": 12,
    "context_length": 18,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_12m_loss = get_performance('ABBV', dates12m, 12)
print(f'ABBV-12m: {abbv_12m_loss}')

100%|██████████| 50/50 [00:00<00:00, 54.69it/s, epoch=30/30, avg_epoch_loss=2.84]

ABBV-12m: 27.61917849429819


STOCKS

In [8]:
tickers = ['ABBV', 'NVS', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG']

In [7]:
stocks = data.copy()

for t in tickers:
  stocks[f'{t}_lag'] = stocks[t].shift(1)
  stocks[f'{t}_lag2'] = stocks[t].shift(2)
  #stocks[f'{t}_lag3'] = stocks[t].shift(3)


stocks['Quater_dummy'] = 0
stocks['Quater_rippel'] = 0
stocks['Date'] = pd.to_datetime(stocks['Date'])
for index, row in stocks.iterrows():
    date = pd.to_datetime(row['Date'])  # Convert the date string to datetime type
    if date.month in [3, 6, 9, 12]:
        stocks.loc[index, 'Quater_dummy'] = 1
    if date.month in [1, 4, 7, 10]:
        stocks.loc[index, 'Quater_rippel'] = 1

stocks = format(stocks)

In [8]:
def data_prep(df, ticker, horizion):
    train_data = ListDataset(
        [
            {
                "start": df.index[0],
                "target": df[ticker].values,
                "dynamic_feat": [
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    test_data = ListDataset(
        [
            {
                "start": df.index[-horizion],
                "target": df[ticker][-horizion:].values,
                "dynamic_feat": [
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    return train_data, test_data

In [13]:
def get_performance(ticker, dates, horizon):

  dta = stocks.copy()
  t_loss = 0

  for d in dates:
    split_date = d
    train_df = dta[dta.index <= split_date]
    test_df = dta[dta.index > split_date]

    train, test = data_prep(train_df, ticker, horizon)
    predictions, check = fit_n_predict(estimator, train, test)
    p95 = np.percentile(predictions, 95, axis=0)
    p50 = np.percentile(predictions, 50, axis=0)
    p5 = np.percentile(predictions, 5, axis=0)

    var  = pd.Series(p5)
    agg = test_df[ticker]
    r = agg.values
    r = r[0:horizon]
    r = pd.Series(r)
    t_loss += tick_loss(0.05, r, var)

  combionation = 13-horizon
  t_loss = t_loss/combionation

  return t_loss

In [14]:
hyperparameters = {
    "prediction_length": 1,
    "context_length": 3,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

# 1month
abbv_1m_loss = get_performance('ABBV', dates1m, 1)
print(f'ABBV-1m: {abbv_1m_loss}')

100%|██████████| 50/50 [00:00<00:00, 109.22it/s, epoch=30/30, avg_epoch_loss=4.89]

ABBV-1m: 1.206717370635937


In [15]:
hyperparameters = {
    "prediction_length": 3,
    "context_length": 4,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_3m_loss = get_performance('ABBV', dates3m, 3)
print(f'ABBV-3m: {abbv_3m_loss}')

100%|██████████| 50/50 [00:00<00:00, 95.33it/s, epoch=30/30, avg_epoch_loss=3.5]

ABBV-3m: 2.4223206913088386


In [16]:
hyperparameters = {
    "prediction_length": 6,
    "context_length": 9,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_6m_loss = get_performance('ABBV', dates6m, 6)
print(f'ABBV-6m: {abbv_6m_loss}')

100%|██████████| 50/50 [00:00<00:00, 55.43it/s, epoch=30/30, avg_epoch_loss=3.46]

ABBV-6m: 6.399264232879345


In [17]:
hyperparameters = {
    "prediction_length": 9,
    "context_length": 12,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_9m_loss = get_performance('ABBV', dates9m, 9)
print(f'ABBV-9m: {abbv_9m_loss}')

100%|██████████| 50/50 [00:01<00:00, 47.64it/s, epoch=30/30, avg_epoch_loss=4.71]

ABBV-9m: 12.207416452942478


In [18]:
hyperparameters = {
    "prediction_length": 12,
    "context_length": 18,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_12m_loss = get_performance('ABBV', dates12m, 12)
print(f'ABBV-12m: {abbv_12m_loss}')

100%|██████████| 50/50 [00:00<00:00, 54.35it/s, epoch=30/30, avg_epoch_loss=1.93]

ABBV-12m: 30.591881175269254


MACRO + STOCKS

In [9]:
all = data.copy()

for m in ['GDP_growth',	'SP500_return',	'inflation_change',	'unemp_change',	'treasury_yield_change']:
  all[f'{m}_lag'] = all[m].shift(1)
  all[f'{m}_lag2'] = all[m].shift(2)
  all[f'{m}_lag3'] = all[m].shift(3)


all['Quater_dummy'] = 0
all['Quater_rippel'] = 0
all['Date'] = pd.to_datetime(all['Date'])
for index, row in all.iterrows():
    date = pd.to_datetime(row['Date'])  # Convert the date string to datetime type
    if date.month in [3, 6, 9, 12]:
        all.loc[index, 'Quater_dummy'] = 1
    if date.month in [1, 4, 7, 10]:
        all.loc[index, 'Quater_rippel'] = 1


for t in tickers:
  all[f'{t}_lag'] = all[t].shift(1)
  all[f'{t}_lag2'] = all[t].shift(2)

all = format(all)

In [10]:
def data_prep(df, ticker, horizon):
    train_data = ListDataset(
        [
            {
                "start": df.index[0],
                "target": df[ticker].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['ABBV_lag'].values,
                              df['ABBV_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    test_data = ListDataset(
        [
            {
                "start": df.index[-horizon],
                "target": df[ticker][-horizon:].values,
                "dynamic_feat": [
                              df['GDP_growth_lag'].values,
                              df['GDP_growth_lag2'].values,
                              df['GDP_growth_lag3'].values,
                              df['SP500_return_lag'].values,
                              df['SP500_return_lag2'].values,
                              df['SP500_return_lag3'].values,
                              df['inflation_change_lag'].values,
                              df['inflation_change_lag2'].values,
                              df['inflation_change_lag3'].values,
                              df['unemp_change_lag'].values,
                              df['unemp_change_lag2'].values,
                              df['unemp_change_lag3'].values,
                              df['treasury_yield_change_lag'].values,
                              df['treasury_yield_change_lag2'].values,
                              df['treasury_yield_change_lag3'].values,
                              df['NVS_lag'].values,
                              df['NVS_lag2'].values,
                              df['AZN_lag'].values,
                              df['AZN_lag2'].values,
                              df['BMY_lag'].values,
                              df['BMY_lag2'].values,
                              df['JNJ_lag'].values,
                              df['JNJ_lag2'].values,
                              df['LLY_lag'].values,
                              df['LLY_lag2'].values,
                              df['MRK_lag'].values,
                              df['MRK_lag2'].values,
                              df['NVO_lag'].values,
                              df['NVO_lag2'].values,
                              df['PFE_lag'].values,
                              df['PFE_lag2'].values,
                              df['ROG_lag'].values,
                              df['ROG_lag2'].values,
                              df['Quater_dummy'].values,
                              df['Quater_rippel'].values
                ]
            }
        ],
        freq="M"
    )

    return train_data, test_data

In [11]:
def get_performance(ticker, dates, horizon):

  dta = all.copy()
  t_loss = 0

  for d in dates:
    split_date = d
    train_df = dta[dta.index <= split_date]
    test_df = dta[dta.index > split_date]

    train, test = data_prep(train_df, ticker, horizon)
    predictions, check = fit_n_predict(estimator, train, test)
    p95 = np.percentile(predictions, 95, axis=0)
    p50 = np.percentile(predictions, 50, axis=0)
    p5 = np.percentile(predictions, 5, axis=0)

    var  = pd.Series(p5)
    agg = test_df[ticker]
    r = agg.values
    r = r[0:horizon]
    r = pd.Series(r)
    t_loss += tick_loss(0.05, r, var)

  combionation = 13-horizon
  t_loss = t_loss/combionation

  return t_loss

In [12]:
hyperparameters = {
    "prediction_length": 1,
    "context_length": 3,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

# 1month
abbv_1m_loss = get_performance('ABBV', dates1m, 1)
print(f'ABBV-1m: {abbv_1m_loss}')

100%|██████████| 50/50 [00:00<00:00, 150.19it/s, epoch=30/30, avg_epoch_loss=4.76]

ABBV-1m: 1.2213121546486223


In [13]:
hyperparameters = {
    "prediction_length": 3,
    "context_length": 4,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

# 1month
abbv_3m_loss = get_performance('ABBV', dates3m, 3)
print(f'ABBV-3m: {abbv_3m_loss}')

100%|██████████| 50/50 [00:00<00:00, 116.23it/s, epoch=30/30, avg_epoch_loss=4.69]

ABBV-3m: 2.234897517452138


In [14]:
hyperparameters = {
    "prediction_length": 6,
    "context_length": 9,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_6m_loss = get_performance('ABBV', dates6m, 6)
print(f'ABBV-6m: {abbv_6m_loss}')

100%|██████████| 50/50 [00:00<00:00, 84.35it/s, epoch=30/30, avg_epoch_loss=3.5]

ABBV-6m: 5.19189998404007


In [15]:
hyperparameters = {
    "prediction_length": 9,
    "context_length": 12,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_9m_loss = get_performance('ABBV', dates9m, 9)
print(f'ABBV-9m: {abbv_9m_loss}')

100%|██████████| 50/50 [00:00<00:00, 69.23it/s, epoch=30/30, avg_epoch_loss=4.73]

ABBV-9m: 10.045787858107683


In [16]:
hyperparameters = {
    "prediction_length": 12,
    "context_length": 18,
    "num_cells": 10,
    "num_layers": 2,
    "dropout_rate": 0.1,
    "batch_size": 1,
    "likelihood": "gaussian",
    "hybridize": True
}
####################################################
estimator = DeepAREstimator(
    prediction_length = hyperparameters["prediction_length"],
    context_length = hyperparameters["context_length"],
    num_cells = hyperparameters["num_cells"],
    num_layers = hyperparameters["num_layers"],
    dropout_rate = hyperparameters["dropout_rate"],
    batch_size = hyperparameters["batch_size"],
    freq = "M",
    trainer = Trainer(epochs = 30)
    )

#3month
abbv_12m_loss = get_performance('ABBV', dates12m, 12)
print(f'ABBV-12m: {abbv_12m_loss}')

100%|██████████| 50/50 [00:00<00:00, 55.60it/s, epoch=30/30, avg_epoch_loss=1.74]

ABBV-12m: 39.28621608786409
